In [1]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 38.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## A base de dados é uma base que mostra noticias e as categorias das noticias

In [2]:
import spacy
nlp = spacy.load('pt_core_news_sm')

In [3]:
import pandas as pd

In [4]:
dados_treino = pd.read_csv('/content/drive/MyDrive/Alura/ML_Avançada/NLP/treino.csv')
dados_teste = pd.read_csv('/content/drive/MyDrive/Alura/ML_Avançada/NLP/teste.csv')

In [5]:
dados_treino.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


###Prever categoria e utilizar o titulo para cirar o modelo word2vec

#Pré Processamento

- Criando um gerador que converte todos os títulos de dados_treino['title'] para letras minúsculas.
- A função trata_textos processa um documento tokenizado (ou seja, uma lista de tokens) e retorna uma string com os tokens válidos unidos por espaços.

In [6]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino['title'])

def trata_textos(doc):
  token_validos = []
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      token_validos.append(token.text)

  if len(token_validos) > 2:
      return " ".join(token_validos)

In [7]:
texto = "Rio de Janeiro é uma##$# 12313 (()) cidade maravilhosa"
doc = nlp(texto)

In [8]:
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

###Tratamento para todos os títulos

Leitura dos textos para tratar em batchs, no nosso caso, o algoritmo lê de mil em mil linhas

In [9]:
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size = 1000, n_process = -1)]

In [10]:
titulos_tratados = pd.DataFrame({'titulo': textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


#Treinamento do modelo cbow

- sg=0:
Define o algoritmo de treinamento. sg=0 usa o algoritmo CBOW (Continuous Bag of Words). Se fosse sg=1, o modelo usaria o algoritmo Skip-gram.
- window=2:
Define o tamanho da janela de contexto. Este parâmetro determina o número de palavras vizinhas consideradas à esquerda e à direita de uma palavra-alvo. Neste caso, a janela de contexto é de 2 palavras.
- vector_size=300:
Define o tamanho dos vetores de palavras. Cada palavra será representada por um vetor de 300 dimensões.
- min_count=5:
Define o limite mínimo de frequência para as palavras. Palavras que aparecem menos de 5 vezes no corpus serão ignoradas.
- alpha=0.03:
Define a taxa de aprendizado inicial. É o valor inicial da taxa de aprendizado usada pelo algoritmo de otimização.
- min_alpha=0.007:
Define a taxa de aprendizado mínima. Durante o treinamento, a taxa de aprendizado decai gradualmente até este valor mínimo.

In [13]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0, window = 2, vector_size = 300, min_count = 5, alpha = 0.03, min_alpha = 0.007)

###Contruindo vocabulario word2vec
- Para construir o vocabulario temos que dropar os valores nulos e duplicados

In [14]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
84466


In [15]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados['titulo']]

In [16]:
w2v_modelo.build_vocab(lista_lista_tokens)

###Treino do modelo com 30 épocas.

In [19]:
w2v_modelo.train(lista_lista_tokens, total_examples=w2v_modelo.corpus_count, epochs = 30)

(14583517, 16207260)

###Testando as palavras mais similares que o nosso modelo considera

In [20]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5694851875305176),
 ('facebook', 0.5088313221931458),
 ('waze', 0.49313652515411377),
 ('uber', 0.47861722111701965),
 ('amazon', 0.4678192138671875),
 ('airbnb', 0.4597543776035309),
 ('canais', 0.4306645691394806),
 ('fbi', 0.4302992522716522),
 ('tesla', 0.4248991906642914),
 ('yahoo', 0.4239816665649414)]

In [21]:
w2v_modelo.wv.most_similar("microsoft")

[('unilever', 0.5639453530311584),
 ('viajante', 0.5350008606910706),
 ('sky', 0.534398078918457),
 ('amazon', 0.5340562462806702),
 ('braskem', 0.5308745503425598),
 ('tesla', 0.5152236819267273),
 ('walmart', 0.5067993402481079),
 ('linkedin', 0.5037802457809448),
 ('pfizer', 0.48962146043777466),
 ('sony', 0.48724105954170227)]

In [22]:
w2v_modelo.wv.most_similar("barcelona")

[('barça', 0.572942316532135),
 ('madrid', 0.5626287460327148),
 ('united', 0.5357410907745361),
 ('leicester', 0.533928632736206),
 ('bayern', 0.5268201231956482),
 ('juventus', 0.5209063291549683),
 ('munique', 0.5187647938728333),
 ('lazio', 0.5149582624435425),
 ('figueirense', 0.5021436810493469),
 ('chelsea', 0.5009716749191284)]

In [23]:
w2v_modelo.wv.most_similar("neymar")

[('messi', 0.47795161604881287),
 ('romário', 0.45622503757476807),
 ('benzema', 0.4331664443016052),
 ('barça', 0.43154746294021606),
 ('cristiane', 0.41826656460762024),
 ('fred', 0.41625210642814636),
 ('suárez', 0.41623246669769287),
 ('psg', 0.40687295794487),
 ('ibrahimovic', 0.40646153688430786),
 ('fernandinho', 0.4019312560558319)]

In [26]:
w2v_modelo.wv.most_similar("bmw")

[('toyota', 0.5695067048072815),
 ('steel', 0.5438239574432373),
 ('tesla', 0.5360438227653503),
 ('renault', 0.5344973206520081),
 ('braskem', 0.5100235939025879),
 ('bélgica', 0.5011749267578125),
 ('reestrutura', 0.4941660463809967),
 ('embraer', 0.49150148034095764),
 ('chrysler', 0.488815575838089),
 ('honda', 0.4830126166343689)]

#Treinamento do modelo SkipGram

In [27]:
w2v_modelo_sg = Word2Vec(sg = 1, window = 5, vector_size = 300, min_count = 5, alpha = 0.03, min_alpha = 0.007)
w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)
w2v_modelo_sg.train(lista_lista_tokens, total_examples=w2v_modelo.corpus_count, epochs = 30)

(14584542, 16207260)

In [28]:
w2v_modelo_sg.wv.most_similar("google")

[('reguladores', 0.4383900761604309),
 ('android', 0.409042090177536),
 ('waze', 0.3890588581562042),
 ('facebook', 0.38825374841690063),
 ('apple', 0.38086363673210144),
 ('patentes', 0.3740023672580719),
 ('yahoo', 0.3591548502445221),
 ('toshiba', 0.35774341225624084),
 ('telefónica', 0.35612040758132935),
 ('anúncios', 0.3550843596458435)]

In [29]:
w2v_modelo_sg.wv.most_similar("microsoft")

[('linkedin', 0.5435748100280762),
 ('chips', 0.5097066760063171),
 ('verizon', 0.4678157866001129),
 ('telefónica', 0.46497559547424316),
 ('fertilizantes', 0.45646148920059204),
 ('silício', 0.4470959007740021),
 ('bitcoin', 0.44526156783103943),
 ('farmacêuticas', 0.4417784512042999),
 ('syngenta', 0.435330331325531),
 ('software', 0.4350524842739105)]

In [30]:
w2v_modelo_sg.wv.most_similar("barcelona")

[('celta', 0.5630292296409607),
 ('espanyol', 0.5102715492248535),
 ('supercopa', 0.4932115972042084),
 ('sevilla', 0.4893488585948944),
 ('villarreal', 0.475085586309433),
 ('athletic', 0.4677700698375702),
 ('wolfsburg', 0.46275776624679565),
 ('monaco', 0.4344217777252197),
 ('bilbao', 0.4333671033382416),
 ('barça', 0.43026888370513916)]

In [31]:
w2v_modelo_sg.wv.most_similar("neymar")

[('barça', 0.5479617118835449),
 ('suárez', 0.48216477036476135),
 ('cavani', 0.4816271960735321),
 ('messi', 0.44334566593170166),
 ('psg', 0.43018123507499695),
 ('cristiane', 0.4131641685962677),
 ('dedada', 0.4090251624584198),
 ('villarreal', 0.4056721031665802),
 ('benzema', 0.398384690284729),
 ('philippe', 0.3979870080947876)]

In [32]:
w2v_modelo_sg.wv.most_similar("bmw")

[('reestrutura', 0.5664843320846558),
 ('audi', 0.5440587997436523),
 ('chrysler', 0.5049031972885132),
 ('intel', 0.49186399579048157),
 ('apreender', 0.48532843589782715),
 ('gm', 0.483180433511734),
 ('toyota', 0.4813898205757141),
 ('airbag', 0.47130122780799866),
 ('honda', 0.46935343742370605),
 ('chips', 0.46150386333465576)]

###Salvando modelos

In [36]:
w2v_modelo.wv.save_word2vec_format('/content/drive/MyDrive/Alura/ML_Avançada/NLP/modelo_cbow.txt', binary = False)
w2v_modelo_sg.wv.save_word2vec_format('/content/drive/MyDrive/Alura/ML_Avançada/NLP/modelo_skipgram.txt', binary = False)